In [50]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time
import pydde as d

In [51]:
#Parameters
samplenum = 4
epochs = 5
hiddenlayers = [90]
input_size = 3
output_size = 3
learning_rate = 0.001
time_length = 5; #seconds

In [57]:
# Generate simulation
dyn = d.PyDyn('Data/point-mass_pendulum.sim', time_length)
#dyn = d.PyDyn('Data/rb-pendulum/twoRb.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)

In [56]:
#Sample targets only variables in z direction
y_target = np.zeros((samplenum,3))
y_target[:,2] = np.random.rand(samplenum)
y_target[:,1] = 2
y_target= torch.tensor(y_target)
y_target

tensor([[0.0000, 2.0000, 0.0763],
        [0.0000, 2.0000, 0.9818],
        [0.0000, 2.0000, 0.3835],
        [0.0000, 2.0000, 0.8418]], dtype=torch.float64)

## Building the custon Simulation activation function and model

In [48]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input.shape}')
        p = input.clone().numpy().transpose()
        y_pred = torch.ones([samplenum,3])
        for i in range(len(p[0, :])):
            state = dyn.compute(p[:,i])
            y_pred[i, :] = torch.tensor(state.y[-3:])
        #print(f'y_pred: {y_pred.shape}')
        
        ctx.save_for_backward(input)
        
        return y_pred
    
    @staticmethod
    def backward(ctx, grad_output):
        print(f'grad output = {grad_output.shape}')
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        dy_dp_batch = torch.ones([3, samplenum*3*time_length])
        for i in range(len(p[0, :])):
            #index = torch.arange(i*3*time_length, i*3*time_length+3*time_length)
            state= dyn.compute(p[:, i])
            dy_dp = dyn.dy_dp(state, p[:, i])
            dy_dp = torch.tensor(dy_dp[-3:, :])
            print(f'dy_dp = {dy_dp.shape}')
            dy_dp_batch[:, (i*3*time_length):(i*3*time_length+(3*time_length))] = dy_dp
        print(f'shape of dy/dp_batch: {dy_dp_batch.shape}')
        
        grad_input = torch.tensor(dy_dp_batch).t().mm(grad_output.t())
        print(f'shape of grad input: {grad_input.shape}')
        return grad_input

Simulate = Simulate.apply

class ActiveLearn(nn.Module):

    def __init__(self, n_in, out_sz):
        super(ActiveLearn, self).__init__()

        self.L_in = nn.Linear(n_in, hiddenlayers[0])
        self.H1 = nn.Linear(hiddenlayers[0], 3*time_length)
        #self.H1 = nn.Linear(hiddenlayers[0], hiddenlayers[1])
        #self.H2 = nn.Linear(hiddenlayers[1], 3*time_length)
        self.P = nn.Linear(3*time_length, 3*time_length)
        self.Relu = nn.ReLU(inplace=True)
    
    def forward(self, input):
        x = self.L_in(input)
        x = self.Relu(x)
        x = self.H1(x)
        x = self.Relu(x)
        #x = self.H2(x)
        #x = self.Relu(x)
        x = self.P(x)
        x = self.Relu(x)
        return x


model = ActiveLearn(input_size, output_size)

criterion = nn.MSELoss()  # RMSE = np.sqrt(MSE)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

y_target = y_target.float()

In [16]:
p_pred = model(y_target)
print(f'input: {p_pred.shape}')
p = p_pred.detach().numpy().transpose()
print(p)
y_pred = torch.ones([samplenum,3])
for i in range(len(p[0, :])):
    state = dyn.compute(p[:,i])
    y_pred[i, :] = torch.tensor(state.y[-3:])
print(f'y_pred: {y_pred}')

input: torch.Size([4, 15])
[[0.         0.         0.         0.        ]
 [0.19863126 0.14947447 0.1638604  0.18559146]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.25616917 0.2295226  0.24310282 0.25499287]
 [0.         0.         0.         0.        ]
 [0.03689428 0.         0.         0.02426878]
 [0.11494696 0.07961529 0.095068   0.11199307]
 [0.08668315 0.10223247 0.10192344 0.09625881]
 [0.         0.         0.         0.        ]
 [0.17958242 0.13337764 0.14984947 0.17250447]
 [0.20030531 0.10532123 0.13499646 0.17711805]
 [0.1168789  0.07053249 0.09257828 0.11364429]
 [0.         0.         0.         0.        ]
 [0.20760596 0.20938584 0.21173942 0.21158448]]
y_pred: tensor([[ 1.4895e-02,  9.8974e-01, -1.6985e-02],
        [ 9.6590e-03,  9.9358e-01, -6.5585e-04],
        [ 1.3374e-02,  9.9232e-01, -3.4950e-03],
        [ 1.5946e-02,  9.9072e-01, -1.0351e-02]])


## Train the model

In [49]:
torch.autograd.set_detect_anomaly(True)

start_time = time.time()

losses = []
y_preds= np.zeros((samplenum, 3))
p_preds= np.zeros((samplenum, 3*time_length))
y_pred = torch.tensor([samplenum,3])
for i in range(epochs):
    p_pred = model(y_target)
    y_pred = Simulate(p_pred)
    #print(f'shape of y_pred: {y_pred.shape}')
    loss = torch.sqrt(criterion(y_pred.float(), y_target)) # RMSE
    #loss = criterion(y_pred.float(), y_truth) + 0.2*(sum(p[0:3]-dyn.p_init[0:3]))**2  # MSE + start condition penalty + p smoothness condition penalty
    #loss =sum((y_pred.float()-y_truth)**2) 
    optimizer.zero_grad()
    #Back Prop
    loss.backward()
    #optimizer.step()
    print(f'epoch: {i:3}/{epochs}  loss: {loss.item():10.8f}')
    #i+=1

print(f'epoch: {i:3} loss: {loss.item():10.8f}') # print the last line
print(f'\nDuration: {(time.time() - start_time)/60:.3f} min') # print the time elapsed

grad output = torch.Size([4, 3])
dy_dp = torch.Size([3, 15])
dy_dp = torch.Size([3, 15])
dy_dp = torch.Size([3, 15])
dy_dp = torch.Size([3, 15])
shape of dy/dp_batch: torch.Size([3, 60])
shape of grad input: torch.Size([60, 4])


RuntimeError: Function SimulateBackward returned an invalid gradient at index 0 - got [60, 4] but expected shape compatible with [4, 15]

In [8]:
#Save Model

if len(losses) == epochs*(samplenum):
    torch.save(model.state_dict(), 'Trained_Model_300420_300s_100e_onlyZpos.pt')
    print('Model saved')
else:
    print('Model has not been trained. Consider loading a trained model instead.')

Model saved


## Test forward propagation

In [36]:
y_target= torch.tensor([0, 2, 0.5])
p = model(y_target)
y_pred = Simulate(p)
y_pred = y_pred.detach().numpy()
p = p.detach().numpy()


yTraj_test = dyn.compute(p)

print(y_pred)
print(y_target)
print(yTraj_test.y[-3:])
print(np.sum(yTraj_test.y[-3:]-y_pred))
print(p)

[0.0944146 1.9530724 0.3841143]
tensor([0.0000, 2.0000, 0.5000])
[0.0944146 1.9530724 0.3841143]
0.0
[0.         0.         0.         0.         0.         0.20432281
 0.         0.         1.451682   1.3087382  0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.6551723  0.         0.         0.
 0.         0.         0.07099061 0.         0.         0.
 0.         0.         0.         0.25406152 0.         0.
 0.         0.         0.         1.1448839  1.0023524  0.
 0.         0.         0.         0.         0.         0.
 0.         0.07773931 0.         0.         0.         0.1156168
 0.         0.         0.         0.         0.         0.
 0.         0.         0.5394917  0.         0.40568525 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.01849897 0.         0.26445943 0.         0.
 0.         0.         0.         0.         0.         1.

## Torch Script Conversion and Saving

In [36]:
input_example = torch.tensor(y_target[0,:])
traced_script_module = torch.jit.trace(model, input_example)
original = model(test_input)

# Test the torch script
test_input= torch.tensor([0, 2, 0.5])
output_example = traced_script_module(test_input)
print(output_example[-12:])
print(original)

tensor([1.9574, 0.0000, 0.0000, 2.7701, 0.0000], grad_fn=<SliceBackward>)


In [ ]:
# Save serialized model
traced_script_module.save("CPP_example_model_latest.pt")

In [49]:
y_pred = torch.ones([samplenum,3])
y_pred

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])